# Projeto Prático 4

Alunos:
- Manoel Victor Florencio de Souza
- Sérgio Alexandre Arruda Pinheiro

In [1]:
import numpy as np
from math import sqrt
from math import ceil
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

import warnings

In [2]:
warnings.simplefilter("ignore")

Carregamento e tratamento do *Dataset*

In [13]:
df = pd.read_csv('seeds_dataset.csv', '\t')
df['type'] = df['type'].map({1:'Kama', 2:'Rosa', 3:'Canadian'})
X = df.iloc[:,:7]
y = df.iloc[:,-1]

In [14]:
df.corr()

,area,perimeter,compactness,length,width,asymmetry,groove
area,1.000000,0.994341,0.608288,0.949985,0.970771,-0.229572,0.863693
perimeter,0.994341,1.000000,0.529244,0.972422,0.944829,-0.217340,0.890784
compactness,0.608288,0.529244,1.000000,0.367915,0.761635,-0.331471,0.226825
length,0.949985,0.972422,0.367915,1.000000,0.860415,-0.171562,0.932806
width,0.970771,0.944829,0.761635,0.860415,1.000000,-0.258037,0.749131
asymmetry,-0.229572,-0.217340,-0.331471,-0.171562,-0.258037,1.000000,-0.011079
groove,0.863693,0.890784,0.226825,0.932806,0.749131,-0.011079,1.000000


In [3]:
df_teste = pd.read_csv('seeds_dataset.csv', '\t')
df_teste = df_teste.drop(['perimeter', 'length','width','groove'], axis=1)
X_teste = df_teste.iloc[:,:-1]
y_teste = df_teste.iloc[:,-1]
df_teste.corr('pearson')

,area,compactness,asymmetry,type
area,1.000000,0.608288,-0.229572,-0.346058
compactness,0.608288,1.000000,-0.331471,-0.531007
asymmetry,-0.229572,-0.331471,1.000000,0.577273
type,-0.346058,-0.531007,0.577273,1.000000


Função **Piramid()** retorna uma lista de tuplas contendo todas as possíveis combinações de neurônios aceitáveis pelo parâmetro **hidden_layer_sizes**  de acordo com a regra da piramide geométrica:
$$ N_{h} = \alpha \cdot \sqrt{N_{i} \cdot N_{o}}$$

In [4]:
def piramid(alpha=2, n_in=7, n_out=1):
    neurons = ceil(alpha * sqrt(n_in * n_out))
    ret = [(neurons,)]
    ret += [(neurons-n, n) for n in range(1, neurons)]
    return ret

Conjunto de parâmetros que irão constituir o *GridSearchCV*

In [5]:
params = {
    'activation': ['logistic', 'relu'], 
    'max_iter': [200, 400, 800],
    'hidden_layer_sizes': list(piramid(alpha=0.5) + piramid(alpha=2) + piramid(alpha=3))
}

In [9]:
params = {
    'activation': ['logistic', 'relu'], 
    'max_iter': [200, 400, 800],
    'hidden_layer_sizes': list(piramid(alpha=0.5, n_in=3) + piramid(alpha=2, n_in=3) + piramid(alpha=3, n_in=3))
}

In [10]:
params

{'activation': ['logistic', 'relu'],
 'max_iter': [200, 400, 800],
 'hidden_layer_sizes': [(1,),
  (4,),
  (3, 1),
  (2, 2),
  (1, 3),
  (6,),
  (5, 1),
  (4, 2),
  (3, 3),
  (2, 4),
  (1, 5)]}

In [11]:
clf = GridSearchCV(MLPClassifier(solver='lbfgs'), params, scoring='f1_macro')

A medida de avaliação do modelo escolhida foi a f1_macro, pois a base de dados é homogênea (possui exatamente 70 exemplos de cada classe) e o problema é multiclasses.

O solver lbfgs foi escolhido pois é o que melhor se adequa em relação ao número de exemplos na base de dados do problema (210) e de acordo com a documentação do sklearn "Para pequenas bases de dados, lbfgs converge mais rapidamente e obtém melhor performance"¹.

Referência: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html ¹

In [ ]:
best_nets = []

In [ ]:
kf = StratifiedKFold(n_splits=3, shuffle=True)
for train_indexes, test_indexes in kf.split(X, y):
    clf.fit(X.iloc[train_indexes], y.iloc[train_indexes])
    print(clf.score(X.iloc[test_indexes], y.iloc[test_indexes]))
    d = clf.best_params_.copy()
    d['score'] = clf.score(X.iloc[test_indexes], y.iloc[test_indexes])
    d['model'] = clf.best_estimator_
    best_nets.append(d)

In [12]:
kf = StratifiedKFold(n_splits=3, shuffle=True)
for train_indexes, test_indexes in kf.split(X_teste, y_teste):
    clf.fit(X_teste.iloc[train_indexes], y_teste.iloc[train_indexes])
    print(clf.score(X_teste.iloc[test_indexes], y_teste.iloc[test_indexes]))
    print(clf.best_params_)

0.8605020022193274
{'activation': 'logistic', 'hidden_layer_sizes': (4,), 'max_iter': 200}
0.8977060311177086
{'activation': 'logistic', 'hidden_layer_sizes': (5, 1), 'max_iter': 800}
0.8674706246134818
{'activation': 'relu', 'hidden_layer_sizes': (6,), 'max_iter': 200}


In [ ]:
best_nets